## Imports

In [ ]:
#!pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    #!pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
 df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = None

# YOUR CODE HERE
get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [7]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [8]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}
title_dictionary

{'Capt': 'Officer',
 'Col': 'Officer',
 'Major': 'Officer',
 'Jonkheer': 'Royalty',
 'Don': 'Royalty',
 'Sir': 'Royalty',
 'Dr': 'Officer',
 'Rev': 'Officer',
 'the Countess': 'Royalty',
 'Mme': 'Mrs',
 'Mlle': 'Miss',
 'Ms': 'Mrs',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Miss': 'Miss',
 'Master': 'Master',
 'Lady': 'Royalty'}

In [9]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df["Title"] = df.Title.map(title_dictionary) 
df_test["Title"] = df_test.Title.map(title_dictionary)

In [10]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [13]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [14]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [36]:
# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [37]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [24]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

In [25]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [26]:
# YOUR CODE HERE
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [27]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [45]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   
    random_state=42,
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:02:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6872505	total: 1.63ms	remaining: 1.62s
1:	learn: 0.6807542	total: 3.06ms	remaining: 1.53s
2:	learn: 0.6752059	total: 4.97ms	remaining: 1.65s
3:	learn: 0.6719610	total: 5.89ms	remaining: 1.47s
4:	learn: 0.6668264	total: 7.43ms	remaining: 1.48s
5:	learn: 0.6614641	total: 9.07ms	remaining: 1.5s
6:	learn: 0.6554242	total: 11.1ms	remaining: 1.58s
7:	learn: 0.6507065	total: 12.3ms	remaining: 1.53s
8:	learn: 0.6454106	total: 13.8ms	remaining: 1.52s
9:	learn: 0.6402708	total: 15.2ms	remaining: 1.5s
10:	learn: 0.6361167	total: 16ms	remaining: 1.44s
11:	learn: 0.6311927	total: 17.6ms	remaining: 1.45s
12:	learn: 0.6263344	total: 19.1ms	remaining: 1.45

273:	learn: 0.3567248	total: 346ms	remaining: 918ms
274:	learn: 0.3563006	total: 348ms	remaining: 917ms
275:	learn: 0.3560017	total: 349ms	remaining: 916ms
276:	learn: 0.3559820	total: 350ms	remaining: 914ms
277:	learn: 0.3558971	total: 351ms	remaining: 912ms
278:	learn: 0.3558614	total: 352ms	remaining: 909ms
279:	learn: 0.3556520	total: 353ms	remaining: 907ms
280:	learn: 0.3554156	total: 354ms	remaining: 906ms
281:	learn: 0.3552727	total: 355ms	remaining: 904ms
282:	learn: 0.3549696	total: 356ms	remaining: 903ms
283:	learn: 0.3549270	total: 357ms	remaining: 900ms
284:	learn: 0.3547521	total: 358ms	remaining: 899ms
285:	learn: 0.3545785	total: 359ms	remaining: 897ms
286:	learn: 0.3544199	total: 361ms	remaining: 897ms
287:	learn: 0.3540908	total: 362ms	remaining: 895ms
288:	learn: 0.3540782	total: 363ms	remaining: 893ms
289:	learn: 0.3538676	total: 364ms	remaining: 891ms
290:	learn: 0.3536366	total: 365ms	remaining: 890ms
291:	learn: 0.3535275	total: 367ms	remaining: 889ms
292:	learn: 

555:	learn: 0.3164490	total: 689ms	remaining: 550ms
556:	learn: 0.3162543	total: 690ms	remaining: 549ms
557:	learn: 0.3160781	total: 692ms	remaining: 548ms
558:	learn: 0.3160735	total: 692ms	remaining: 546ms
559:	learn: 0.3159351	total: 693ms	remaining: 545ms
560:	learn: 0.3158968	total: 694ms	remaining: 543ms
561:	learn: 0.3157431	total: 696ms	remaining: 542ms
562:	learn: 0.3156623	total: 697ms	remaining: 541ms
563:	learn: 0.3155756	total: 698ms	remaining: 540ms
564:	learn: 0.3154482	total: 699ms	remaining: 538ms
565:	learn: 0.3152751	total: 700ms	remaining: 537ms
566:	learn: 0.3151786	total: 702ms	remaining: 536ms
567:	learn: 0.3150918	total: 703ms	remaining: 534ms
568:	learn: 0.3150108	total: 704ms	remaining: 533ms
569:	learn: 0.3149472	total: 705ms	remaining: 532ms
570:	learn: 0.3147643	total: 706ms	remaining: 531ms
571:	learn: 0.3147473	total: 707ms	remaining: 529ms
572:	learn: 0.3146449	total: 709ms	remaining: 528ms
573:	learn: 0.3145541	total: 710ms	remaining: 527ms
574:	learn: 

808:	learn: 0.2907126	total: 1.03s	remaining: 243ms
809:	learn: 0.2905846	total: 1.03s	remaining: 241ms
810:	learn: 0.2904460	total: 1.03s	remaining: 240ms
811:	learn: 0.2902295	total: 1.03s	remaining: 239ms
812:	learn: 0.2902002	total: 1.03s	remaining: 238ms
813:	learn: 0.2901156	total: 1.03s	remaining: 236ms
814:	learn: 0.2897777	total: 1.03s	remaining: 235ms
815:	learn: 0.2895513	total: 1.04s	remaining: 234ms
816:	learn: 0.2893932	total: 1.04s	remaining: 233ms
817:	learn: 0.2892635	total: 1.04s	remaining: 231ms
818:	learn: 0.2892567	total: 1.04s	remaining: 230ms
819:	learn: 0.2891635	total: 1.04s	remaining: 229ms
820:	learn: 0.2889824	total: 1.04s	remaining: 227ms
821:	learn: 0.2888882	total: 1.04s	remaining: 226ms
822:	learn: 0.2887538	total: 1.04s	remaining: 225ms
823:	learn: 0.2887309	total: 1.05s	remaining: 223ms
824:	learn: 0.2886909	total: 1.05s	remaining: 222ms
825:	learn: 0.2886051	total: 1.05s	remaining: 221ms
826:	learn: 0.2884566	total: 1.05s	remaining: 220ms
827:	learn: 

,Model,Accuracy,Bal Acc.,Time
1,Decision Tree,83.798883,82.496706,0.011969
2,CatBoost,83.240223,80.961792,1.463812
3,Random Forest,82.681564,80.777339,0.140654
4,Extra Trees,82.122905,80.322793,0.122673
5,XGBoost,81.564246,80.678524,0.082778
6,Skl GBM,81.005587,78.333333,0.087777
7,Skl HistGBM,81.005587,78.333333,0.090761
8,LightGBM,79.888268,78.774704,0.047871
9,AdaBoost,79.329609,78.860343,0.069824


In [46]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [47]:
# YOUR CODE HERE
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(
    n_splits= 10,
    shuffle=True,
    random_state = 42
)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = cross_val_predict(model, x_val, y_val, cv=skf)
    pred = model.predict(x_val) # PASSES ASSERT BUT WRONG!
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: Use

[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: Use

[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: Use

...

303:	learn: 0.3841690	total: 240ms	remaining: 550ms
304:	learn: 0.3838594	total: 241ms	remaining: 550ms
305:	learn: 0.3837233	total: 242ms	remaining: 549ms
306:	learn: 0.3833015	total: 243ms	remaining: 548ms
307:	learn: 0.3828609	total: 244ms	remaining: 548ms
308:	learn: 0.3826188	total: 244ms	remaining: 546ms
309:	learn: 0.3820222	total: 245ms	remaining: 546ms
310:	learn: 0.3814147	total: 246ms	remaining: 545ms
311:	learn: 0.3809945	total: 247ms	remaining: 544ms
312:	learn: 0.3803742	total: 247ms	remaining: 543ms
313:	learn: 0.3798981	total: 248ms	remaining: 542ms
314:	learn: 0.3794426	total: 250ms	remaining: 543ms
315:	learn: 0.3789897	total: 251ms	remaining: 544ms
316:	learn: 0.3786849	total: 252ms	remaining: 543ms
317:	learn: 0.3784463	total: 253ms	remaining: 542ms
318:	learn: 0.3781229	total: 253ms	remaining: 541ms
319:	learn: 0.3777787	total: 254ms	remaining: 540ms
320:	learn: 0.3773389	total: 255ms	remaining: 539ms
321:	learn: 0.3772021	total: 256ms	remaining: 538ms
322:	learn: 

508:	learn: 0.3275325	total: 404ms	remaining: 389ms
509:	learn: 0.3272823	total: 405ms	remaining: 389ms
510:	learn: 0.3271103	total: 405ms	remaining: 388ms
511:	learn: 0.3269268	total: 406ms	remaining: 387ms
512:	learn: 0.3267983	total: 407ms	remaining: 386ms
513:	learn: 0.3266465	total: 407ms	remaining: 385ms
514:	learn: 0.3264941	total: 408ms	remaining: 385ms
515:	learn: 0.3262865	total: 410ms	remaining: 384ms
516:	learn: 0.3260786	total: 411ms	remaining: 384ms
517:	learn: 0.3257477	total: 411ms	remaining: 383ms
518:	learn: 0.3256697	total: 412ms	remaining: 382ms
519:	learn: 0.3255539	total: 413ms	remaining: 381ms
520:	learn: 0.3253132	total: 413ms	remaining: 380ms
521:	learn: 0.3250765	total: 414ms	remaining: 379ms
522:	learn: 0.3247853	total: 415ms	remaining: 378ms
523:	learn: 0.3245943	total: 415ms	remaining: 377ms
524:	learn: 0.3243459	total: 416ms	remaining: 377ms
525:	learn: 0.3239752	total: 417ms	remaining: 376ms
526:	learn: 0.3238064	total: 418ms	remaining: 375ms
527:	learn: 

713:	learn: 0.2891904	total: 565ms	remaining: 226ms
714:	learn: 0.2890670	total: 566ms	remaining: 226ms
715:	learn: 0.2888752	total: 568ms	remaining: 225ms
716:	learn: 0.2886137	total: 569ms	remaining: 225ms
717:	learn: 0.2882599	total: 570ms	remaining: 224ms
718:	learn: 0.2880657	total: 571ms	remaining: 223ms
719:	learn: 0.2880331	total: 571ms	remaining: 222ms
720:	learn: 0.2880147	total: 571ms	remaining: 221ms
721:	learn: 0.2877490	total: 572ms	remaining: 220ms
722:	learn: 0.2876938	total: 573ms	remaining: 219ms
723:	learn: 0.2875756	total: 573ms	remaining: 219ms
724:	learn: 0.2873598	total: 574ms	remaining: 218ms
725:	learn: 0.2872639	total: 575ms	remaining: 217ms
726:	learn: 0.2870692	total: 576ms	remaining: 216ms
727:	learn: 0.2869310	total: 576ms	remaining: 215ms
728:	learn: 0.2868035	total: 577ms	remaining: 215ms
729:	learn: 0.2867132	total: 578ms	remaining: 214ms
730:	learn: 0.2866515	total: 578ms	remaining: 213ms
731:	learn: 0.2865378	total: 579ms	remaining: 212ms
732:	learn: 

916:	learn: 0.2602010	total: 729ms	remaining: 66ms
917:	learn: 0.2600548	total: 730ms	remaining: 65.2ms
918:	learn: 0.2599822	total: 731ms	remaining: 64.5ms
919:	learn: 0.2597500	total: 732ms	remaining: 63.7ms
920:	learn: 0.2597190	total: 733ms	remaining: 62.9ms
921:	learn: 0.2595809	total: 734ms	remaining: 62.1ms
922:	learn: 0.2593222	total: 735ms	remaining: 61.3ms
923:	learn: 0.2590679	total: 736ms	remaining: 60.5ms
924:	learn: 0.2590289	total: 736ms	remaining: 59.7ms
925:	learn: 0.2588218	total: 737ms	remaining: 58.9ms
926:	learn: 0.2587454	total: 738ms	remaining: 58.1ms
927:	learn: 0.2586346	total: 739ms	remaining: 57.3ms
928:	learn: 0.2585571	total: 739ms	remaining: 56.5ms
929:	learn: 0.2583481	total: 740ms	remaining: 55.7ms
930:	learn: 0.2581824	total: 742ms	remaining: 55ms
931:	learn: 0.2580556	total: 744ms	remaining: 54.3ms
932:	learn: 0.2577721	total: 744ms	remaining: 53.5ms
933:	learn: 0.2576592	total: 745ms	remaining: 52.7ms
934:	learn: 0.2574716	total: 746ms	remaining: 51.9

,Model,Accuracy,Bal Acc.,Time
1,Random Forest,83.240223,81.772069,0.149569
2,CatBoost,83.240223,80.961792,1.527903
3,Decision Tree,82.681564,81.587615,0.012965
4,XGBoost,81.564246,80.678524,0.080750
5,Skl GBM,81.005587,78.333333,0.086768
6,Skl HistGBM,81.005587,78.333333,0.090795
7,Extra Trees,80.446927,79.229249,0.123636
8,LightGBM,79.888268,78.774704,0.044224
9,AdaBoost,79.329609,78.860343,0.070809


In [48]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [39]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data


# YOUR CODE HERE
best_model = tree_classifiers["Skl HistGBM"]
best_model.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [40]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

In [41]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [42]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"